In [1]:
 # Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key

In [2]:
#Loading the csv exported in Part I to a DataFrame
cities_df=pd.read_csv("../output_data/Cities.csv")
cities_df=cities_df.drop(columns=["Unnamed: 0"])
cities_df.head()

,City,Lat,Lng,Max temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,mataura,-46.1927,168.8643,55.47,88,100,5.82,NZ,1638655653
1,tasiilaq,65.6145,-37.6368,28.56,88,100,38.68,GL,1638655653
2,chokurdakh,70.6333,147.9167,-40.54,100,59,2.95,RU,1638655654
3,rikitea,-23.1203,-134.9692,76.06,71,40,10.00,PF,1638655654
4,hermanus,-34.4187,19.2345,63.90,90,100,17.65,ZA,1638655655


<h2> Humidity Heatmap</h2>

In [3]:
#Configure gmaps.
gmaps.configure(api_key=g_key)
#Use the Lat and Lng as locations and Humidity as the weight.
#Add Heatmap layer to map.

# Fill NaN values and convert to float
cities_df.dropna(inplace=True)

In [4]:
# Storing latitude and longitude in locations dataframe
locations = cities_df[["Lat","Lng"]]
locations.head()

,Lat,Lng
0,-46.1927,168.8643
1,65.6145,-37.6368
2,70.6333,147.9167
3,-23.1203,-134.9692
4,-34.4187,19.2345


In [5]:
humidity=cities_df["Humidity"].astype(float)
humidity.head()

0     88.0
1     88.0
2    100.0
3     71.0
4     90.0
Name: Humidity, dtype: float64

In [6]:
# Plot Heatmap
fig = gmaps.figure(map_type='HYBRID')
heatmap_layer = gmaps.heatmap_layer(locations,weights=humidity)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

<h2>new DataFrame fitting weather criteria</h2>

In [7]:
ideal_weather=cities_df.loc[cities_df["Max temp"]<80]
ideal_weather=ideal_weather.loc[ideal_weather["Max temp"]>70]
ideal_weather=ideal_weather.loc[ideal_weather["Wind Speed"]<10]
ideal_weather=ideal_weather.loc[ideal_weather["Cloudiness"]==0]
ideal_weather

,City,Lat,Lng,Max temp,Humidity,Cloudiness,Wind Speed,Country,Date
39,biloela,-24.4167,150.5000,77.95,70,0,3.98,AU,1638655666
69,saint-pierre,-21.3393,55.4781,76.64,77,0,4.61,RE,1638655674
200,lahij,13.1667,44.5833,70.30,66,0,3.56,YE,1638655714
273,perth,-31.9333,115.8333,70.09,59,0,5.75,AU,1638655737
323,ampanihy,-24.7000,44.7500,74.68,59,0,3.36,MG,1638655749
367,blythe,33.6103,-114.5964,77.04,13,0,4.68,US,1638655761
497,beloha,-25.1667,45.0500,73.71,78,0,1.23,MG,1638655821


<h2>Hotel Map</h2>

In [8]:
ideal_weather["Hotel name"]=""
ideal_weather["Hotel lat"]=""
ideal_weather["Hotel lon"]=""

In [9]:
 # params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    "key": g_key
}

hotel_name =[]
for index, row in ideal_weather.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    # changing location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel = requests.get(base_url, params=params)
    hotel=hotel.json()
    ideal_weather.loc[index,"Hotel name"]=hotel["results"][0]["name"]
    ideal_weather.loc[index,"Hotel lat"]=hotel["results"][0]["geometry"]["location"]["lat"]
    ideal_weather.loc[index,"Hotel lon"]=hotel["results"][0]["geometry"]["location"]["lng"]

In [10]:
ideal_weather

,City,Lat,Lng,Max temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel name,Hotel lat,Hotel lon
39,biloela,-24.4167,150.5000,77.95,70,0,3.98,AU,1638655666,Biloela,-24.402535,150.512432
69,saint-pierre,-21.3393,55.4781,76.64,77,0,4.61,RE,1638655674,Saint-Paul,-21.009882,55.269867
200,lahij,13.1667,44.5833,70.30,66,0,3.56,YE,1638655714,Al Habilayn,13.532666,44.848211
273,perth,-31.9333,115.8333,70.09,59,0,5.75,AU,1638655737,Perth,-31.952312,115.861309
323,ampanihy,-24.7000,44.7500,74.68,59,0,3.36,MG,1638655749,Ampanihy,-24.693049,44.745176
367,blythe,33.6103,-114.5964,77.04,13,0,4.68,US,1638655761,Blythe,33.617773,-114.588261
497,beloha,-25.1667,45.0500,73.71,78,0,1.23,MG,1638655821,Marolinta,-25.138769,44.641239


In [39]:
hotel_coord=ideal_weather[["Hotel lat", "Hotel lon"]]

hotel_layer = gmaps.symbol_layer(
    hotel_coord, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=5,
    info_box_content=[f"Name: {hotel}" for hotel in ideal_weather["Hotel name"]]
)
fig = gmaps.figure()
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [34]:
# Create a combined map
fig = gmaps.figure()

fig.add_layer(heatmap_layer)
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))